# Python-Oracle 연동 - 데이터 조회

### [1] 패키지 참조

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import cx_Oracle as cx
from sqlalchemy import create_engine
from pandas import DataFrame, read_sql_table

## #02. cx_oracle을 사용한 연동

데이터베이스 연동 과정은 `데이터베이스 접속 -> SQL 실행 객체(cursor) 생성 -> 결과 처리 -> 데이터베이스 접속 해제`의 순서로 진행된다.

### [1] 데이터베이스 접속

In [ ]:
dbcon = cx.connect("C##",                
                   " ",                
                   "localhost:1521/xe"  
)
dbcon

### [2] Cursor 객체 생성

In [3]:
cursor = dbcon.cursor()

### [3] 데이터 조회를 위한 SQL문 수행

#### (1) 튜플을 원소로 갖는 리스트 형태로 조회

In [4]:
sql = "SELECT * FROM JOBS"

try:
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)
except Exception as e:
    print("데이터 조회 실패", e)

[('AD_PRES', 'President', 20000, 40000), ('AD_VP', 'Administration Vice President', 15000, 30000), ('AD_ASST', 'Administration Assistant', 3000, 6000), ('FI_MGR', 'Finance Manager', 8200, 16000), ('FI_ACCOUNT', 'Accountant', 4200, 9000), ('AC_MGR', 'Accounting Manager', 8200, 16000), ('AC_ACCOUNT', 'Public Accountant', 4200, 9000), ('SA_MAN', 'Sales Manager', 10000, 20000), ('SA_REP', 'Sales Representative', 6000, 12000), ('PU_MAN', 'Purchasing Manager', 8000, 15000), ('PU_CLERK', 'Purchasing Clerk', 2500, 5500), ('ST_MAN', 'Stock Manager', 5500, 8500), ('ST_CLERK', 'Stock Clerk', 2000, 5000), ('SH_CLERK', 'Shipping Clerk', 2500, 5500), ('IT_PROG', 'Programmer', 4000, 10000), ('MK_MAN', 'Marketing Manager', 9000, 15000), ('MK_REP', 'Marketing Representative', 4000, 9000), ('HR_REP', 'Human Resources Representative', 4000, 9000), ('PR_REP', 'Public Relations Representative', 4500, 10500)]


#### (2) 딕셔너리를 원소로 갖는 리스트 형태로 조회

cx_oracle은 별도의 딕셔너리 형태를 제공하지 않는다.

cursor객체의 rowfactory 프로퍼티를 직접 재정의 해야 한다.

In [5]:
sql = "SELECT * FROM JOBS"

try:
    cursor.execute(sql)

    # rowfactory 재정의 - 접속 해제 전까지 최초 1회만 수행하면 된다.
    cursor.rowfactory = lambda *args: dict(zip([d[0] for d in cursor.description], args))

    result = cursor.fetchall()
    print(result)
except Exception as e:
    print("데이터 조회 실패", e)

[{'JOB_ID': 'AD_PRES', 'JOB_TITLE': 'President', 'MIN_SALARY': 20000, 'MAX_SALARY': 40000}, {'JOB_ID': 'AD_VP', 'JOB_TITLE': 'Administration Vice President', 'MIN_SALARY': 15000, 'MAX_SALARY': 30000}, {'JOB_ID': 'AD_ASST', 'JOB_TITLE': 'Administration Assistant', 'MIN_SALARY': 3000, 'MAX_SALARY': 6000}, {'JOB_ID': 'FI_MGR', 'JOB_TITLE': 'Finance Manager', 'MIN_SALARY': 8200, 'MAX_SALARY': 16000}, {'JOB_ID': 'FI_ACCOUNT', 'JOB_TITLE': 'Accountant', 'MIN_SALARY': 4200, 'MAX_SALARY': 9000}, {'JOB_ID': 'AC_MGR', 'JOB_TITLE': 'Accounting Manager', 'MIN_SALARY': 8200, 'MAX_SALARY': 16000}, {'JOB_ID': 'AC_ACCOUNT', 'JOB_TITLE': 'Public Accountant', 'MIN_SALARY': 4200, 'MAX_SALARY': 9000}, {'JOB_ID': 'SA_MAN', 'JOB_TITLE': 'Sales Manager', 'MIN_SALARY': 10000, 'MAX_SALARY': 20000}, {'JOB_ID': 'SA_REP', 'JOB_TITLE': 'Sales Representative', 'MIN_SALARY': 6000, 'MAX_SALARY': 12000}, {'JOB_ID': 'PU_MAN', 'JOB_TITLE': 'Purchasing Manager', 'MIN_SALARY': 8000, 'MAX_SALARY': 15000}, {'JOB_ID': 'PU_CL

### [4] 조회 결과를 활용한 후속 처리

#### (1) 수업에서 배운 내용 활용

In [10]:
with open('department.csv', 'w', encoding='utf-8') as f:
    for i, v in enumerate(result):
        if i == 0:
            keys = list(v.keys())
            titleLine = "%s\n" % ",".join(keys)
            f.write(titleLine)
        
        values = list(v.values())
        # values의 모든 원소를 str 타입으로 변경
        values_str = list(map(str, values))
        
        valueLine = "%s\n" % ",".join(values_str)
        f.write(valueLine)

#### (2) 데이터프레임

In [7]:
df = DataFrame(result)
df

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20000,40000
1,AD_VP,Administration Vice President,15000,30000
2,AD_ASST,Administration Assistant,3000,6000
3,FI_MGR,Finance Manager,8200,16000
4,FI_ACCOUNT,Accountant,4200,9000
5,AC_MGR,Accounting Manager,8200,16000
6,AC_ACCOUNT,Public Accountant,4200,9000
7,SA_MAN,Sales Manager,10000,20000
8,SA_REP,Sales Representative,6000,12000
9,PU_MAN,Purchasing Manager,8000,15000


In [11]:
df.to_csv("department2.csv", index=False, encoding='utf-8')

In [12]:
df.to_excel("department3.xlsx", index=False)

### [5] 데이터베이스 접속 해제


In [13]:
try:
    cursor.close()
    dbcon.close()
except Exception as e:
    print("접속 상태가 아닙니다.", e)

## #03. sqlalchemy를 사용한 연동

데이터베이스의 종류를 구분하지 않고 공통된 연동 방법을 제공하는 인터페이스

### [1] 데이터베이스 접속하기

#### (1) 접속 문자열 생성

`oracle+cx_oracle://계정이름:비밀번호@접속주소/SID`

In [23]:
conStr = "oracle+cx_oracle://C##:@localhost:1521/xe"

#### (2) 접속

In [24]:
engine = create_engine(conStr)
dbcon = engine.connect()

### [2] 데이터 조회하기

조회 결과를 pandas의 DataFrame객체 형태로 반환한다.

#### (1) 특정 테이블의 모든 데이터 조회

In [ ]:
df = read_sql_table("REGIONS", con=dbcon)
df

#### (2) 원하는 컬럼만 조회하기

In [27]:
df = read_sql_table('student', columns=['studno', 'name', 'grade', 'idnum'], con=dbcon)
df

,studno,name,grade,idnum


### [3] 데이터베이스 접속 해제

In [ ]:
dbcon.close()